In [1]:
import urllib

import numpy as np
import tensorflow as tf
tf.enable_v2_behavior()

from rcnn_sat import preprocess_image
from rcnn_sat.bl_net import bl_net

# Restore model and extract activations
This example shows how to extract activations from a pre-trained model. We'll use the BL model trained on eco-set.

## Building the model

First, we build the graph for the model using a placeholder for the input.

In [2]:
input_layer = tf.keras.layers.Input((128, 128, 3))
model = bl_net(input_layer, classes=565, cumulative_readout=True)

Instructions for updating:
Colocations handled automatically by placer.


All model functions take an input layer as the first argument and the number of classes as the second. Note for ecoset `classes=565` and for ImageNet `classes=1000`.

BL has an additional argument specifying whether or not to use a cumulative readout. By deafult `cumulative_readout=False`.

## Loading the weights
Download the pre-trained weights for the models from OSF (weights for other models can be found [here](https://osf.io/mz9hw/)).

In [3]:
_, msg = urllib.request.urlretrieve(
    'https://osf.io/9td5p/download', 'bl_ecoset.h5')
print(msg)

Server: nginx
Date: Fri, 10 May 2019 09:11:49 GMT
Content-Type: application/octet-stream
Content-Length: 112560228
Content-Disposition: attachment; filename="bl_ecoset.h5"; filename*=UTF-8''bl_ecoset.h5
X-Waterbutler-Request-Id: d9e83d84-eb01-4da8-803e-dec5ed80955a
Cache-Control: no-cache, no-store, max-age=0, must-revalidate
Expires: Mon, 01 Jan 1990 00:00:00 GMT
Pragma: no-cache
Via: 1.1 google
Alt-Svc: clear
Connection: close




Load the weights into the model

In [4]:
model.load_weights('bl_ecoset.h5')

## Extract activations

To get the activation in Keras we need to know the name of the layer. The name of the output at each layer and time step has the format `ReLU_Layer_{layer_index}_Time_{time_index}`. So, the output of the first layer on the third time step is labelled `ReLU_Layer_0_Time_2` (note that zero-indexing is used).

We use this to create a function mapping from input to output that we call to get a numpy array of the layer activations.

In [5]:
get_layer_activation = tf.keras.backend.function(
    [model.input],
    [model.get_layer('ReLU_Layer_0_Time_2').output])

Now we'll get the activations based on a random image. We scale the image into the correct range for the pre-trained networks by passing the image through `preprocess_image`. The image must be in `uint8` format (values in the range `[0, 255]`).

In [6]:
random_img = np.random.randint(0, 256, (1, 128, 128, 3), dtype=np.uint8)
preprocessed_img = preprocess_image(random_img)
get_layer_activation(preprocessed_img)

[array([[[[0.        , 0.        , 0.        , ..., 0.20635995,
           1.0930816 , 0.5146626 ],
          [0.92382973, 0.1553492 , 0.04881164, ..., 0.42562833,
           0.35726875, 0.14409715],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.2982943 , 0.24723515],
          ...,
          [0.        , 0.        , 0.        , ..., 0.        ,
           2.3006628 , 0.        ],
          [0.        , 0.        , 0.13931724, ..., 0.        ,
           1.5231669 , 0.10467729],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ]],
 
         [[0.        , 0.        , 0.        , ..., 0.00790107,
           0.        , 0.7055516 ],
          [3.1366339 , 1.4123831 , 0.        , ..., 0.        ,
           0.34940588, 0.5651195 ],
          [0.        , 0.33985692, 0.        , ..., 0.        ,
           0.        , 0.        ],
          ...,
          [0.        , 0.02948871, 0.        , ..., 0.        ,
   